# Google Gemini SDK Introduction

This notebook demonstrates how to use the Google Gemini AI SDK to interact with the Gemini model in both text and audio modes.

## Setup
First, we'll install the required package and initialize the client with our API key.

In [1]:
!pip install -U -q google-genai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### NB: Output audio transcription for Vertex requires `google-genai=1.11.0` !!

See also: https://github.com/googleapis/python-genai/releases/tag/v1.11.0

In [2]:
!pip show google-genai

Name: google-genai
Version: 1.11.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: Apache-2.0
Location: /Users/heikohotz/Projects/gemini-multimodal-live-dev-guide/.venv/lib/python3.13/site-packages
Requires: anyio, google-auth, httpx, pydantic, requests, typing-extensions, websockets
Required-by: 


In [3]:
from google import genai

## Text Interaction Example

Below we'll demonstrate how to have a text conversation with Gemini. The code:
1. Sets up a configuration for text responses
2. Opens an async connection to the model
3. Sends a message and receives the response in chunks
4. Prints each chunk of the response as it arrives

In [4]:
PROJECT_ID = "<YOUR_GOOGLE_CLOUD_PROJECT_ID>"
LOCATION = "us-central1"

In [6]:
model_id = "gemini-2.0-flash-live-preview-04-09"

client = genai.Client(vertexai=True, location=LOCATION, project=PROJECT_ID)
config = {"response_modalities": ["TEXT"]}

async with client.aio.live.connect(model=model_id, config=config) as session:
    message = "Hello? Gemini, are you there?"
    print("> ", message, "\n")
    await session.send_client_content(turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True)

    async for response in session.receive():
        print(response.text)

>  Hello? Gemini, are you there? 

Yes
, I'
m here. What can I do for you?

None
None


## Audio Generation Example

Now we'll see how to generate audio responses from Gemini. This section:
1. Creates a wave file handler to save the audio
2. Configures the model for audio output
3. Sends a text prompt and receives audio data
4. Saves the audio chunks and plays them in the notebook

Note: Make sure your browser's audio is enabled to hear the responses.

In [7]:
import contextlib
import wave


@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

In [8]:
from IPython.display import display, Audio

config = {"response_modalities": ["AUDIO"]}


async with client.aio.live.connect(model=model_id, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send_client_content(turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True)

    first = True
    async for response in session.receive():
      if response.data is not None:
        model_turn = response.server_content.model_turn
        if first:
          print(model_turn.parts[0].inline_data.mime_type)
          first = False
        print('.', end='.')
        wav.writeframes(response.data)


display(Audio(file_name, autoplay=True))

>  Hello? Gemini are you there? 

audio/pcm
..........................

# Text and Audio Interaction Example

This section demonstrates how to receive both text and audio responses from Gemini. The code:
1. Sets up a configuration for text and audio responses
2. Opens an async connection to the model
3. Sends a message and receives the responses in chunks
4. Prints each text chunk of the response as it arrives
5. Collects the audio chunks and plays them in the notebook




In [9]:
from IPython.display import display, Audio

config = {
    "response_modalities": ["AUDIO"],
    "output_audio_transcription": {}, # Enable output transcription
}

async with client.aio.live.connect(model=model_id, config=config) as session:
  file_name = 'audio.wav'
  with wave_file(file_name) as wav:
    transcription_chunk_count = 0
    transcription_complete = ""

    message = "Hello? Gemini are you there?"
    print("> ", message, "\n")
    await session.send_client_content(
        turns={"role": "user", "parts": [{"text": message}]}, turn_complete=True
    )
    first = True
    async for response in session.receive():
        if (response.server_content is not None and
            response.server_content.output_transcription is not None and
            response.server_content.output_transcription.text is not None):
            transcription_chunk_count += 1
            # Access the text attribute of the Transcription object
            transcription_text = response.server_content.output_transcription.text
            print(f"Transcription Chunk {transcription_chunk_count}: {transcription_text}")
            transcription_complete += transcription_text
        if response.data is not None:
            model_turn = response.server_content.model_turn
            if first:
                print(model_turn.parts[0].inline_data.mime_type)
                first = False
            print('.', end='.')
            wav.writeframes(response.data)



display(Audio(file_name, autoplay=True))
print(f"Full Transcription:\n{transcription_complete.strip()}")


>  Hello? Gemini are you there? 

Transcription Chunk 1: Yes
Transcription Chunk 2: , I'
Transcription Chunk 3: m here. What
Transcription Chunk 4:  can I do
Transcription Chunk 5:  for you?
audio/pcm
..........................

Full Transcription:
Yes, I'm here. What can I do for you?
